
# Fraud Notes Classifier (LLM Embeddings → XGBoost)
This notebook:
- loads `fraud_notes_synth.csv` (columns: **text**, **label**)  
- stratified split (train / val / test)  
- encodes notes with a pre-trained LLM encoder (`distilbert-base-uncased`)  
- trains **XGBoost**  
- evaluates (report, confusion matrix, accuracy, ROC-AUC)  
- runs quick inference and (optional) saves artifacts


In [2]:

# Run once if needed
!pip install -U sentence-transformers xgboost scikit-learn pandas numpy joblib


  Using cached numpy-2.3.4-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 302.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 2.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2

[notice] A new release of 

In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sentence_transformers import SentenceTransformer
from xgboost import XGBClassifier

SEED = 42
np.random.seed(SEED)


In [4]:

csv_path = "fraud_notes_synth.csv"  
df = pd.read_csv(csv_path)
df = df.dropna(subset=["text", "label"]).reset_index(drop=True)
df["text"] = df["text"].astype(str).str.strip()
print(df.shape)
df.head()


(600, 2)


,text,label
0,Customer acknowledges they shared credentials ...,first_party
1,Caller states phone number and email on file a...,third_party
2,Victim reports an unknown account opened using...,third_party
3,Client acknowledges mailing mailing address an...,first_party
4,Customer claims identity theft; FTC affidavit ...,third_party


In [5]:

train_df, test_df = train_test_split(
    df, test_size=0.20, random_state=SEED, stratify=df["label"]
)

train_df, val_df = train_test_split(
    train_df, test_size=0.10, random_state=SEED, stratify=train_df["label"]
)

len(train_df), len(val_df), len(test_df)


(432, 48, 120)

In [6]:

le = LabelEncoder()
y_train = le.fit_transform(train_df["label"])
y_val   = le.transform(val_df["label"])
y_test  = le.transform(test_df["label"])

classes = list(le.classes_)
n_classes = len(classes)
classes


['first_party', 'third_party']

In [ ]:

#embedder_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder_name = "distilbert-base-uncased"
embedder = SentenceTransformer(embedder_name)


No sentence-transformers model found with name distilbert-base-uncased. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:

X_train = embedder.encode(train_df["text"].tolist(), normalize_embeddings=True, show_progress_bar=True)
X_val   = embedder.encode(val_df["text"].tolist(),   normalize_embeddings=True, show_progress_bar=True)
X_test  = embedder.encode(test_df["text"].tolist(),  normalize_embeddings=True, show_progress_bar=True)

X_train.shape, X_val.shape, X_test.shape


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

((432, 768), (48, 768), (120, 768))

In [8]:

params = {
    "n_estimators": 1000,
    "max_depth": 6,
    "learning_rate": 0.05,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "random_state": SEED,
    "tree_method": "hist",
    "n_jobs": -1,
    "early_stopping_rounds": 50
}

if n_classes == 2:
    params.update({"objective": "binary:logistic", "eval_metric": "logloss"})
else:
    params.update({"objective": "multi:softprob", "num_class": n_classes, "eval_metric": "mlogloss"})

model = XGBClassifier(**params)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=False
)
model


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,50
,enable_categorical,False
,eval_metric,'logloss'


In [9]:

y_prob = model.predict_proba(X_test)
y_pred = y_prob.argmax(axis=1)

print(classification_report(y_test, y_pred, target_names=classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy:", round(accuracy_score(y_test, y_pred), 4))

if n_classes == 2:
    print("ROC-AUC:", round(roc_auc_score(y_test, y_prob[:, 1]), 4))


              precision    recall  f1-score   support

 first_party       1.00      1.00      1.00        60
 third_party       1.00      1.00      1.00        60

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

Confusion matrix:
 [[60  0]
 [ 0 60]]
Accuracy: 1.0
ROC-AUC: 1.0


In [10]:

peek = test_df.copy().reset_index(drop=True).head(10)
peek_emb = embedder.encode(peek["text"].tolist(), normalize_embeddings=True)
peek_prob = model.predict_proba(peek_emb)
peek_pred = peek_prob.argmax(axis=1)
peek["pred_label"] = le.inverse_transform(peek_pred)
peek[["text", "label", "pred_label"]]


,text,label,pred_label
0,Borrower confirms they applied for the payment...,first_party,first_party
1,Member used their own card and now claims serv...,first_party,first_party
2,Card was shipped to an address the customer do...,third_party,third_party
3,Victim states stolen wallet; merchant attempte...,third_party,third_party
4,Customer claims identity theft; FTC affidavit ...,third_party,third_party
5,Customer claims identity theft; FTC affidavit ...,third_party,third_party
6,Cardholder confirms prior relationship with me...,first_party,first_party
7,Multiple accounts opened the same day with mis...,third_party,third_party
8,Caller states phone number and email on file a...,third_party,third_party
9,Account takeover alert: password reset via unr...,third_party,third_party


In [11]:

note = "Caller states phone number and email on file are unfamiliar; SIM swap suspected."
emb = embedder.encode([note], normalize_embeddings=True)
prob = model.predict_proba(emb)[0]
pred = le.inverse_transform([prob.argmax()])[0]
pred, dict(zip(classes, map(float, prob)))


('third_party',
 {'first_party': 0.003063678741455078, 'third_party': 0.9969363212585449})

In [15]:

import joblib, os
os.makedirs("artifacts", exist_ok=True)
_ = joblib.dump(model, "artifacts/xgb_model.joblib")
_ = joblib.dump(le,    "artifacts/label_encoder.joblib")
with open("artifacts/embedder.txt", "w") as f:
    f.write(embedder_name)
"saved."


'saved.'

In [ ]:

import joblib
from sentence_transformers import SentenceTransformer

loaded_model = joblib.load("artifacts/xgb_model.joblib")
loaded_le    = joblib.load("artifacts/label_encoder.joblib")
with open("artifacts/embedder.txt") as f:
    loaded_embedder_name = f.read().strip()
loaded_embedder = SentenceTransformer(loaded_embedder_name)

note2 = "Customer admits they opened the account and later disputed charges."
e2 = loaded_embedder.encode([note2], normalize_embeddings=True)
p2 = loaded_model.predict_proba(e2)[0]
pred2 = loaded_le.inverse_transform([p2.argmax()])[0]
pred2, dict(zip(list(loaded_le.classes_), map(float, p2)))
